In [1]:
import os
import numpy as np

import itk
from itk import TubeTK as ttk
import SimpleITK as sitk

from itkwidgets import view

In [2]:
main_fld = '/media/peirong/PR/IXI/IXI-Vessel'
pre_fld = os.path.join(main_fld, 'level-0')
cur_fld = os.path.join(main_fld, 'level-1')

main_fld = '/home/peirong/Desktop'

def cropping(img, tol = 0, crop_range_lst = None, save_path = None):
    
    '''
    img: itk readable image
    crop_range_lst: [[x0, y0, z0], [x1, y1, z1]]
    '''

    orig_nda = sitk.GetArrayFromImage(img)
    
    if crop_range_lst is None:
        
        if len(orig_nda.shape) > 3: # 4D data: last axis (t=0) as time dimension
            nda = orig_nda[..., 0]
        else:
            nda = np.copy(orig_nda)

        # Mask of non-black pixels (assuming image has a single channel).
        mask = nda > tol

        # Coordinates of non-black pixels.
        coords = np.argwhere(mask)

        # Bounding box of non-black pixels.
        #x0, y0, z0 = coords.min(axis=0) + 1   # Abandom 1 layer
        #x1, y1, z1 = coords.max(axis=0) - 10  # Abandom 11 layers
        x0, y0, z0 = coords.min(axis=0)
        x1, y1, z1 = coords.max(axis=0) + 1   # slices are exclusive at the top

        # Check the the bounding box #
        print('    Cropping Slice  [%d, %d)' % (x0, x1))
        print('    Cropping Row    [%d, %d)' % (y0, y1))
        print('    Cropping Column [%d, %d)' % (z0, z1))
        
    else:
        
        [[x0, y0, z0], [x1, y1, z1]] = crop_range_lst

    cropped_nda = orig_nda[x0 : x1, y0 : y1, z0 : z1]
    cropped_img = sitk.GetImageFromArray(cropped_nda, isVector = len(orig_nda.shape) > 3)
    #new_origin = [img.GetOrigin()[0] + img.GetSpacing()[0] * z0,\
      #  img.GetOrigin()[1] + img.GetSpacing()[1] * y0,\
        #    img.GetOrigin()[2] + img.GetSpacing()[2] * x0]  # numpy reverse to sitk
    #cropped_img.SetOrigin(new_origin)
    cropped_img.SetOrigin(img.GetOrigin())
    cropped_img.SetSpacing(img.GetSpacing())
    cropped_img.SetDirection(img.GetDirection())
    
    if save_path:
        sitk.WriteImage(cropped_img, save_path)
    return cropped_img, [[x0, y0, z0], [x1, y1, z1]] #, new_origin
        
        
    


In [26]:
main_fld = '/home/peirong/Desktop'
mask_path = os.path.join(main_fld, 'MRAMask-Brain.mha')
mask_img = sitk.ReadImage(mask_path)
cropped_mask_img, crop_ranges = cropping(mask_img)

mask_path = '%s_cropped%s' % (mask_path[:-4], mask_path[-4:])
sitk.WriteImage(cropped_mask_img, mask_path)

def crop_n_save(img_path, crop_ranges, save_path = None):
    cropped_img, _ = cropping(sitk.ReadImage(img_path), crop_range_lst = crop_ranges)
    save_path = '%s_cropped%s' % (img_path[:-4], img_path[-4:]) if save_path is None else save_path
    print(save_path)
    sitk.WriteImage(cropped_img, save_path)
    return save_path

mra_iso_path = crop_n_save(os.path.join(main_fld, 'MRA-Iso.mha'), crop_ranges)
mra_brain_path = crop_n_save(os.path.join(main_fld, 'MRA-Brain.mha'), crop_ranges)
t1_iso_path = crop_n_save(os.path.join(main_fld, 'T1_2MRA-Iso.mha'), crop_ranges)
t2_iso_path = crop_n_save(os.path.join(main_fld, 'T2_2MRA-Iso.mha'), crop_ranges)
t1_brain_path = crop_n_save(os.path.join(main_fld, 'T1_2MRA-Brain.mha'), crop_ranges)
t2_brain_path = crop_n_save(os.path.join(main_fld, 'T2_2MRA-Brain.mha'), crop_ranges)

crop_n_save(os.path.join(main_fld, 'DWI0_2MRA-Brain.mha'), crop_ranges)
crop_n_save(os.path.join(main_fld, 'DWI1_2MRA-Brain.mha'), crop_ranges)
crop_n_save(os.path.join(main_fld, 'DWI2_2MRA-Brain.mha'), crop_ranges)
crop_n_save(os.path.join(main_fld, 'DWI3_2MRA-Brain.mha'), crop_ranges)
crop_n_save(os.path.join(main_fld, 'DWI4_2MRA-Brain.mha'), crop_ranges)
crop_n_save(os.path.join(main_fld, 'DWI5_2MRA-Brain.mha'), crop_ranges)

    Cropping Slice  [0, 80)
    Cropping Row    [16, 181)
    Cropping Column [49, 192)
/home/peirong/Desktop/MRA-Iso_cropped.mha
/home/peirong/Desktop/MRA-Brain_cropped.mha
/home/peirong/Desktop/T1_2MRA-Iso_cropped.mha
/home/peirong/Desktop/T2_2MRA-Iso_cropped.mha
/home/peirong/Desktop/T1_2MRA-Brain_cropped.mha
/home/peirong/Desktop/T2_2MRA-Brain_cropped.mha
/home/peirong/Desktop/DWI0_2MRA-Brain_cropped.mha
/home/peirong/Desktop/DWI1_2MRA-Brain_cropped.mha
/home/peirong/Desktop/DWI2_2MRA-Brain_cropped.mha
/home/peirong/Desktop/DWI3_2MRA-Brain_cropped.mha
/home/peirong/Desktop/DWI4_2MRA-Brain_cropped.mha
/home/peirong/Desktop/DWI5_2MRA-Brain_cropped.mha


'/home/peirong/Desktop/DWI5_2MRA-Brain_cropped.mha'

In [29]:
ImageType = itk.Image[itk.F, 3]
ReaderType = itk.ImageFileReader[ImageType]
ResampleType = ttk.ResampleImage[ImageType]

def read_img(img_path):
    reader = ReaderType.New(FileName = img_path)
    reader.Update()
    return reader.GetOutput()

#NOTE: This assumed the data has been resampled into isotropic spacing, registered and brain-stripped #
im_mra_iso = read_img(mra_iso_path)
im_mra_brain = read_img(mra_brain_path)
im_t1_iso  = read_img(t1_iso_path)
im_t1_brain = read_img(t1_brain_path)
im_t2_iso  = read_img(t2_iso_path)
im_t2_brain = read_img(t2_brain_path)

'''reader = ReaderType.New(FileName = os.path.join(pre_fld, "MRA-Brain.mha"))
reader.Update()
im1Brain = reader.GetOutput()

reader = ReaderType.New(FileName = os.path.join(pre_fld, "MRT1-Iso.mha"))
reader.Update()
im2iso = reader.GetOutput()
reader = ReaderType.New(FileName = os.path.join(pre_fld, "MRT1-Brain.mha"))
reader.Update()
im2Brain = reader.GetOutput()

reader = ReaderType.New(FileName = os.path.join(pre_fld, "MRT2-Iso.mha"))
reader.Update()
im3iso = reader.GetOutput()
reader = ReaderType.New(FileName = os.path.join(pre_fld, "MRT2-Brain.mha"))
reader.Update()
im3Brain = reader.GetOutput()'''

'reader = ReaderType.New(FileName = os.path.join(pre_fld, "MRA-Brain.mha"))\nreader.Update()\nim1Brain = reader.GetOutput()\n\nreader = ReaderType.New(FileName = os.path.join(pre_fld, "MRT1-Iso.mha"))\nreader.Update()\nim2iso = reader.GetOutput()\nreader = ReaderType.New(FileName = os.path.join(pre_fld, "MRT1-Brain.mha"))\nreader.Update()\nim2Brain = reader.GetOutput()\n\nreader = ReaderType.New(FileName = os.path.join(pre_fld, "MRT2-Iso.mha"))\nreader.Update()\nim3iso = reader.GetOutput()\nreader = ReaderType.New(FileName = os.path.join(pre_fld, "MRT2-Brain.mha"))\nreader.Update()\nim3Brain = reader.GetOutput()'

In [33]:
#view(im_mra_iso)
#view(im_t1_iso)
#view(im_t1_brain)

In [35]:
imMath = ttk.ImageMath[ImageType,ImageType].New()
imMath.SetInput(im_mra_brain)
imMath.Blur(1.0)
imBlur = imMath.GetOutput()
imBlurArray = itk.GetArrayViewFromImage(imBlur)

numSeeds = 10
seedCoverage = 20
seedCoord = np.zeros([numSeeds,3])
for i in range(numSeeds):
    seedCoord[i] = np.unravel_index(np.argmax(imBlurArray, axis=None), imBlurArray.shape)
    indx = [int(seedCoord[i][0]),int(seedCoord[i][1]),int(seedCoord[i][2])]
    minX = max(indx[0]-seedCoverage,0)
    maxX = max(indx[0]+seedCoverage,imBlurArray.shape[0])
    minY = max(indx[1]-seedCoverage,0)
    maxY = max(indx[1]+seedCoverage,imBlurArray.shape[1])
    minZ = max(indx[2]-seedCoverage,0)
    maxZ = max(indx[2]+seedCoverage,imBlurArray.shape[2])
    imBlurArray[minX:maxX,minY:maxY,minZ:maxZ]=0
    indx.reverse()
    seedCoord[:][i] = im_mra_brain.TransformIndexToPhysicalPoint(indx)
#print(seedCoord)

In [41]:
# Manually extract a few vessels to form an image-specific training set
vSeg = ttk.SegmentTubes[ImageType].New()
#vSeg.SetInput(im_mra_iso) # im_mra_iso: non-skull-stripped (Same result as input im_mra_brain)
vSeg.SetInput(im_mra_brain)
vSeg.SetVerbose(True)
vSeg.SetMinRoundness(0.1)
vSeg.SetMinCurvature(0.001)
vSeg.SetRadiusInObjectSpace( 1 )
for i in range(numSeeds):
    print("**** Processing seed " + str(i) + " : " + str(seedCoord[i]))
    vSeg.ExtractTubeInObjectSpace( seedCoord[i], i )
    
tubeMaskImage = vSeg.GetTubeMaskImage()

**** Processing seed 0 : [-65.0739212   -4.62519836  20.85701728]
**** Processing seed 1 : [-53.0739212   16.37480164   5.85701728]
**** Processing seed 2 : [-91.0739212    4.37480164  36.85701728]
**** Processing seed 3 : [-78.0739212   16.37480164   5.85701728]
**** Processing seed 4 : [-49.0739212   76.37480164   0.85701728]
**** Processing seed 5 : [-92.0739212   65.37480164   0.85701728]
**** Processing seed 6 : [-113.0739212    19.37480164   46.85701728]
**** Processing seed 7 : [-56.0739212   97.37480164  65.85701728]
**** Processing seed 8 : [-113.0739212    49.37480164   51.85701728]
**** Processing seed 9 : [-71.0739212   87.37480164  37.85701728]


In [42]:
#view(tubeMaskImage)

In [43]:
LabelMapType = itk.Image[itk.UC,3]

trMask = ttk.ComputeTrainingMask[ImageType,LabelMapType].New()
trMask.SetInput( tubeMaskImage )
trMask.SetGap( 3 )
#trMask.SetObjectWidth( 1 )
trMask.SetNotObjectWidth( 1 )
trMask.Update()
fgMask = trMask.GetOutput()

In [45]:
#view(fgMask)

In [52]:
enhancer = ttk.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
enhancer.SetInput( im_mra_iso ) # ? why use non-skull-stripped version ?
enhancer.AddInput( im_t1_iso )
enhancer.AddInput( im_t2_iso )
enhancer.SetLabelMap( fgMask )
enhancer.SetRidgeId( 255 )
enhancer.SetBackgroundId( 127 ) #128
enhancer.SetUnknownId( 0 )
enhancer.SetTrainClassifier(True)
enhancer.SetUseIntensityOnly(True)
enhancer.SetScales([0.3333,1,2.5])
enhancer.Update()
enhancer.ClassifyImages()

In [55]:
mra_vess = itk.SubtractImageFilter( Input1=enhancer.GetClassProbabilityImage(0), Input2=enhancer.GetClassProbabilityImage(1))
itk.imwrite( mra_vess, os.path.join(main_fld, "MRA-VesselEnhanced_ISO.mha"), compression=True)

brainMask = itk.imread( os.path.join(main_fld, "MRAMask-Brain_cropped.mha"), itk.F )
MRABrainVess = itk.MultiplyImageFilter(Input1 = mra_vess, Input2=brainMask)
itk.imwrite( MRABrainVess, os.path.join(main_fld, "MRA-Brain_cropped-VesselEnhanced_ISO.mha"), compression=True)

In [50]:
#view(enhancer.GetOutput())
view(enhancer.GetClassProbabilityImage(0))
#imMath = ttk.ImageMath[ImageType,ImageType].New(Input = segmenter.GetClassProbabilityImage(0))
#imMath.AddImages( segmenter.GetClassProbabilityImage(1), 1, -1 )
#view(imMath.GetOutput())

In [56]:
enhancer = ttk.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
enhancer.SetInput( im_mra_brain )
enhancer.AddInput( im_t1_brain )
enhancer.AddInput( im_t2_brain )
enhancer.SetLabelMap( fgMask )
enhancer.SetRidgeId( 255 )
enhancer.SetBackgroundId( 127 ) #128
enhancer.SetUnknownId( 0 )
enhancer.SetTrainClassifier(True)
enhancer.SetUseIntensityOnly(True)
enhancer.SetScales([0.3333,1,2.5])
enhancer.Update()
enhancer.ClassifyImages()

In [57]:
mra_vess = itk.SubtractImageFilter( Input1=enhancer.GetClassProbabilityImage(0), Input2=enhancer.GetClassProbabilityImage(1))
itk.imwrite( mra_vess, os.path.join(main_fld, "MRA-VesselEnhanced_Brain.mha"), compression=True)

brainMask = itk.imread( os.path.join(main_fld, "MRAMask-Brain_cropped.mha"), itk.F )
MRABrainVess = itk.MultiplyImageFilter(Input1 = mra_vess, Input2=brainMask)
itk.imwrite( MRABrainVess, os.path.join(main_fld, "MRA-Brain_cropped-VesselEnhanced_Brain.mha"), compression=True)

In [51]:
#view(enhancer.GetOutput())
view(enhancer.GetClassProbabilityImage(0))
#imMath = ttk.ImageMath[ImageType,ImageType].New(Input = segmenter.GetClassProbabilityImage(0))
#imMath.AddImages( segmenter.GetClassProbabilityImage(1), 1, -1 )
#view(imMath.GetOutput())

In [23]:
#itk.imwrite( enhancer.GetClassProbabilityImage(0), os.path.join(cur_fld, "MRA-VesselProb.mha"), compression=True)
#itk.imwrite( enhancer.GetClassProbabilityImage(1), os.path.join(cur_fld, "MRA-NotVesselProb.mha"), compression=True)

In [22]:

im1vess = itk.SubtractImageFilter( Input1=enhancer.GetClassProbabilityImage(0), Input2=enhancer.GetClassProbabilityImage(1))
itk.imwrite( im1vess, os.path.join(cur_fld, "MRA-VesselEnhanced.mha"), compression=True)

brainMask = itk.imread( os.path.join(pre_fld, "MRMask-Brain.mha"), itk.F )
im1BrainVess = itk.MultiplyImageFilter(Input1 = im1vess, Input2=brainMask)
itk.imwrite( im1BrainVess, os.path.join(cur_fld, "MRA-Brain-VesselEnhanced.mha"), compression=True)